# Analyse complète : Autonomie réelle vs WLTP

Cette analyse examine en profondeur la relation entre l'autonomie réelle des véhicules électriques et leur autonomie WLTP annoncée.

## Structure Notebook

**Analyses descriptives**
   - Moyenne et distribution du ratio autonomie réelle / WLTP
   - Comparaison entre marques et modèles
   - Corrélations (Pearson, Spearman)
   - Visualisations (histogrammes, boxplots, violin plots)
**Facteurs d'influence**
   - Influence de la température
   - Influence de l'âge et du SoH de la batterie
**Analyses comparatives**
   - Par marque et modèle
   - Par technologie (type de batterie, capacité)


## 1. Préparation des données


In [ ]:
# Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from core.s3.s3_utils import S3Service
from core.spark_utils import create_spark_session
from core.pandas_utils import series_start_end_diff
from core.sql_utils import get_sqlalchemy_engine, text
import os
import warnings
warnings.filterwarnings('ignore')

# Configuration de l'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [ ]:
# Connexion aux services
s3 = S3Service()
spark_session = create_spark_session(os.environ.get('S3_KEY'), os.environ.get('S3_SECRET'))
engine = get_sqlalchemy_engine()
con = engine.connect()


In [ ]:
# Chargement des données véhicules avec informations complètes
query = """
SELECT 
    vd.*,
    v.vin,
    vm.model_name,
    vm.type as version,
    vm.autonomy as wltp_range,
    b.battery_chemistry,
    b.capacity,
    b.net_capacity,
    o.oem_name as make
FROM vehicle_data vd
JOIN vehicle v ON v.id = vd.vehicle_id
JOIN vehicle_model vm ON vm.id = v.vehicle_model_id
JOIN battery b ON b.id = vm.battery_id
JOIN oem o ON o.id = vm.oem_id
"""

with engine.connect() as connection:
    vehicle_df = pd.read_sql(text(query), connection)

vehicle_df['vin'] = vehicle_df['vin'].astype('category').astype(str)
vehicle_df.sort_values('timestamp', inplace=True)
vehicle_df['timestamp'] = pd.to_datetime(vehicle_df['timestamp'])
vehicle_df['soh'] = vehicle_df['soh'].fillna(vehicle_df['soh_oem'])
print(f"Données véhicules chargées : {len(vehicle_df)} lignes")
print(f"Nombre de VINs uniques : {vehicle_df['vin'].nunique()}")


In [ ]:

tss_mercedes = s3.read_parquet_df_spark(spark_session, 'processed_phases/processed_phases_mercedes_benz.parquet').toPandas()
tss_stellantis = s3.read_parquet_df_spark(spark_session, 'processed_phases/processed_phases_stellantis.parquet').toPandas()
tss_bmw = s3.read_parquet_df_spark(spark_session, 'processed_phases/processed_phases_bmw.parquet').toPandas()
tss_kia = s3.read_parquet_df_spark(spark_session, 'processed_phases/processed_phases_kia.parquet').toPandas()
tss_renault = s3.read_parquet_df_spark(spark_session, 'processed_phases/processed_phases_renault.parquet').toPandas()
tss_volkswagen = s3.read_parquet_df_spark(spark_session, 'processed_phases/processed_phases_volkswagen.parquet').toPandas()
tss_tesla = s3.read_parquet_df_spark(spark_session, 'processed_phases/processed_phases_tesla_fleet_telemetry.parquet').toPandas()

all_tss = pd.concat([tss_mercedes, tss_stellantis, tss_bmw, tss_kia, tss_volkswagen, tss_renault, tss_tesla], ignore_index=True)
# all_tss = tss_stellantis.copy()



In [ ]:
all_tss.head()

In [ ]:
# Filtrer les périodes de décharge
tss_discharge = all_tss[all_tss['SOC_DIFF'] < 0].copy()


In [ ]:
df = tss_discharge.merge(vehicle_df[['vin', 'model_name', 'version', 'wltp_range', 'battery_chemistry', 'capacity', 'net_capacity', 'make']], left_on='VIN', right_on='vin', how='left')

In [ ]:
# Fusion avec les données véhicules
df = pd.merge_asof(
    tss_discharge.sort_values('DATETIME_END'), 
    vehicle_df[['timestamp', 'vin', 'soh', 'model_name', 'version', 'wltp_range', 
                'battery_chemistry', 'capacity', 'net_capacity', 'make']], 
    left_on='DATETIME_END', 
    right_on='timestamp', 
    left_by='VIN', 
    right_by='vin', 
    direction='nearest',
    suffixes=('', '_vehicle')
)




In [ ]:
# Calcul des métriques clés

# Autonomie réelle (km par 100% de SoC)
df['ODOMETER_DIFF'] = df['ODOMETER_LAST'] - df['ODOMETER_FIRST']
df['km_per_soc'] = (df['ODOMETER_DIFF'] / df['SOC_DIFF']).abs()
df['supposed_autonomy'] = df['km_per_soc'] * 100  # Autonomie pour 100% de charge

# Ratio autonomie réelle / WLTP (en %)
df['ratio_supposed_wltp'] = (df['supposed_autonomy'] / df['wltp_range']) * 100

# # Forward-fill du SoH par véhicule
# df['soh'] = df.groupby('vin')['soh'].transform(lambda x: x.ffill())

# #  Autonomie WLTP ajustée par SoH
# df['wltp_adjusted_soh'] = df['wltp_range'] * df['soh']

# Saison 
df['month'] = df['DATETIME_BEGIN'].dt.month
df['season'] = df['month'].map({
    12: 'Hiver', 1: 'Hiver', 2: 'Hiver',
    3: 'Printemps', 4: 'Printemps', 5: 'Printemps',
    6: 'Été', 7: 'Été', 8: 'Été',
    9: 'Automne', 10: 'Automne', 11: 'Automne'
})

In [ ]:
# Filtrage des valeurs aberrantes
# On garde les ratios entre 30% et 130% (autonomie réelle entre 30% et 130% du WLTP)
# Au moins 5% de décharge
df_clean = df[
    (df['ratio_supposed_wltp'] > 30) & 
    (df['ratio_supposed_wltp'] < 130) &
    (df['supposed_autonomy'] > 0) &
    (df['SOC_DIFF'].abs() > 5) 
].copy()

print(f"Données nettoyées : {len(df_clean)} phases ({len(df_clean)/len(df)*100:.1f}% des données)")
print(f"Nombre de véhicules : {df_clean['vin'].nunique()}")
print(f"Nombre de marques : {df_clean['make'].nunique()}")
print(f"Nombre de modèles : {df_clean['model_name'].nunique()}")


In [ ]:
df_vin = df_clean.groupby('vin', as_index=False).agg({
    'ratio_supposed_wltp': 'median',
    'supposed_autonomy': 'median',
    'wltp_range': 'first',
    'capacity': 'first',
    'net_capacity': 'first',
    'battery_chemistry': 'first',
    'km_per_soc': 'median',
    'make': 'first',
    'model_name': 'first',
    'version': 'first',
    'ODOMETER_FIRST': 'median',
    })

## 2. Analyses descriptives


### 2.1 Distribution globale du ratio Autonomie réelle / WLTP


In [ ]:
# Statistiques globales
print("=== RATIO AUTONOMIE RÉELLE / WLTP ===")
print(f"Moyenne : {df_vin['ratio_supposed_wltp'].mean():.1f}%")
print(f"Médiane : {df_vin['ratio_supposed_wltp'].median():.1f}%")
print(f"Écart-type : {df_vin['ratio_supposed_wltp'].std():.1f}%")
print(f"\nPercentiles :")
print(f"  10% : {df_vin['ratio_supposed_wltp'].quantile(0.10):.1f}%")
print(f"  25% : {df_vin['ratio_supposed_wltp'].quantile(0.25):.1f}%")
print(f"  75% : {df_vin['ratio_supposed_wltp'].quantile(0.75):.1f}%")
print(f"  90% : {df_vin['ratio_supposed_wltp'].quantile(0.90):.1f}%")


In [ ]:
# Histogramme de la distribution
fig = px.histogram(
    df_clean,
    x='ratio_supposed_wltp',
    nbins=50,
    title='Distribution du ratio Autonomie constatée / WLTP',
    labels={'ratio_supposed_wltp': 'Ratio Autonomie constatée / WLTP (%)'},
    color_discrete_sequence=['#636EFA']
)

# Ajouter ligne verticale pour la moyenne
mean_ratio = df_clean['ratio_supposed_wltp'].mean()
fig.add_vline(x=mean_ratio, line_dash="dash", line_color="red", 
              annotation_text=f"Moyenne: {mean_ratio:.1f}%")
fig.add_vline(x=100, line_dash="dash", line_color="green", 
              annotation_text="WLTP (100%)")

fig.update_layout(
    xaxis_title="Ratio (%)",
    yaxis_title="Nombre de phases de décharge",
    height=500
)

fig.show()

fig.write_html("graph/hist_ratio_wltp_per_charge.html")


In [ ]:
# Histogramme de la distribution
fig = px.histogram(
    df_vin,
    x='ratio_supposed_wltp',
    nbins=50,
    title='Distribution du ratio Autonomie constatée / WLTP',
    labels={'ratio_supposed_wltp': 'Ratio Autonomie constatée / WLTP (%)'},
    color_discrete_sequence=['#636EFA']
)

# Ajouter ligne verticale pour la moyenne
mean_ratio = df_vin['ratio_supposed_wltp'].mean()
fig.add_vline(x=mean_ratio, line_dash="dash", line_color="red", 
              annotation_text=f"Moyenne: {mean_ratio:.1f}%")
fig.add_vline(x=100, line_dash="dash", line_color="green", 
              annotation_text="WLTP (100%)")

fig.update_layout(
    xaxis_title="Ratio (%)",
    yaxis_title="Nombre de vehciules",
    height=500
)

fig.show()

fig.write_html("graph/hist_ratio_wltp_per_vin.html")


### 2.2 Comparaison par marque


In [ ]:
# Statistiques par marque
stats_by_make = df_clean.groupby('make').agg({
    'ratio_supposed_wltp': ['mean', 'median', 'std', 'count'],
    'supposed_autonomy': 'mean',
    'wltp_range': 'mean',
    'vin': 'nunique'
}).round(1)

stats_by_make.columns = ['_'.join(col).strip() for col in stats_by_make.columns.values]
stats_by_make = stats_by_make.rename(columns={
    'ratio_supposed_wltp_mean': 'Ratio moyen (%)',
    'ratio_supposed_wltp_median': 'Ratio médian (%)',
    'ratio_supposed_wltp_std': 'Écart-type (%)',
    'ratio_supposed_wltp_count': 'Nb phases',
    'supposed_autonomy_mean': 'Autonomie réelle (km)',
    'wltp_range_mean': 'WLTP moyen (km)',
    'vin_nunique': 'Nb véhicules'
})

stats_by_make = stats_by_make.sort_values('Ratio moyen (%)', ascending=False)
print("\n=== STATISTIQUES PAR MARQUE ===")
print(stats_by_make)


In [ ]:
# Statistiques par marque
stats_by_make = df_vin.groupby('make').agg({
    'ratio_supposed_wltp': ['mean', 'median', 'std', 'count'],
    'supposed_autonomy': 'mean',
    'wltp_range': 'mean',
    #'vin': 'nunique'
}).round(1)

stats_by_make.columns = ['_'.join(col).strip() for col in stats_by_make.columns.values]
stats_by_make = stats_by_make.rename(columns={
    'ratio_supposed_wltp_mean': 'Ratio moyen (%)',
    'ratio_supposed_wltp_median': 'Ratio médian (%)',
    'ratio_supposed_wltp_std': 'Écart-type (%)',
    'ratio_supposed_wltp_count': 'Nb vehicules',
    'supposed_autonomy_mean': 'Autonomie réelle (km)',
    'wltp_range_mean': 'WLTP moyen (km)',
    #'vin_nunique': 'Nb véhicules'
})

stats_by_make = stats_by_make.sort_values('Ratio moyen (%)', ascending=False)
print("\n=== STATISTIQUES PAR MARQUE ===")
print(stats_by_make)


In [ ]:
# Boxplot par marque
fig = px.box(
    df_vin,
    x='make',
    y='ratio_supposed_wltp',
    title='Distribution du ratio Autonomie réelle / WLTP par marque',
    labels={'make': 'Marque', 'ratio_supposed_wltp': 'Ratio (%)'},
    color='make'
)

fig.add_hline(y=100, line_dash="dash", line_color="gray", 
              annotation_text="WLTP (100%)")

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    showlegend=False
)

fig.show()
fig.write_html("graph/boxplot_ratio_wltp_per_make.html")

In [ ]:
# Violin plot par marque
fig = px.violin(
    df_vin,
    x='make',
    y='ratio_supposed_wltp',
    title='Distribution du ratio Autonomie réelle / WLTP par marque (Violin plot)',
    labels={'make': 'Marque', 'ratio_supposed_wltp': 'Ratio (%)'},
    color='make',
    box=True,
    points='outliers'
)

fig.add_hline(y=100, line_dash="dash", line_color="gray")

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    showlegend=False
)

fig.show()


### 2.3 Comparaison par modèle


In [ ]:
# Top 15 modèles les plus représentés
top_models = df_vin['model_name'].value_counts().head(10).index
df_top_models = df_vin[df_vin['model_name'].isin(top_models)]

# Statistiques par modèle
stats_by_model = df_top_models.groupby('model_name').agg({
    'ratio_supposed_wltp': ['mean', 'median', 'std', 'count'],
    'supposed_autonomy': 'mean',
    'wltp_range': 'mean',
    'make': 'first',
    'vin': 'nunique'
}).round(1)

stats_by_model.columns = ['_'.join(col).strip() if col[1] else col[0] for col in stats_by_model.columns.values]
stats_by_model = stats_by_model.sort_values('ratio_supposed_wltp_mean', ascending=False)

print("\n=== TOP 15 MODÈLES - STATISTIQUES ===")
stats_by_model


In [ ]:
# Boxplot pour les top modèles
fig = px.box(
    df_top_models,
    x='model_name',
    y='ratio_supposed_wltp',
    title='Ratio Autonomie réelle / WLTP - Top 15 modèles',
    labels={'model_name': 'Modèle', 'ratio_supposed_wltp': 'Ratio (%)'},
    color='make'
)

fig.add_hline(y=100, line_dash="dash", line_color="gray", 
              annotation_text="WLTP (100%)")

fig.update_layout(
    xaxis_tickangle=-45,
    height=600
)

fig.show()


### Correlation

In [ ]:
# Scatter plots pour les corrélations importantes
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'WLTP vs Autonomie réelle',
        'SoH vs Ratio réel/WLTP',
        'Kilométrage vs Ratio réel/WLTP',
        'Capacité batterie vs Ratio réel/WLTP'
    ]
)

# WLTP vs supposed autonomy
sample = df_clean.sample(n=min(5000, len(df_clean)), random_state=42)
fig.add_trace(
    go.Scatter(x=sample['wltp_range'], y=sample['supposed_autonomy'], 
               mode='markers', marker=dict(size=3, opacity=0.3),
               name=''),
    row=1, col=1
)

# # SoH vs Ratio
# fig.add_trace(
#     go.Scatter(x=sample['soh'], y=sample['ratio_supposed_wltp'], 
#                mode='markers', marker=dict(size=3, opacity=0.3),
#                name=''),
#     row=1, col=2
# )

# Odometer vs Ratio
fig.add_trace(
    go.Scatter(x=sample['ODOMETER_FIRST'], y=sample['ratio_supposed_wltp'], 
               mode='markers', marker=dict(size=3, opacity=0.3),
               name=''),
    row=2, col=1
)

# Capacity vs Ratio
if 'capacity' in sample.columns:
    fig.add_trace(
        go.Scatter(x=sample['capacity'], y=sample['ratio_supposed_wltp'], 
                   mode='markers', marker=dict(size=3, opacity=0.3),
                   name=''),
        row=2, col=2
    )

fig.update_xaxes(title_text="WLTP (km)", row=1, col=1)
# fig.update_xaxes(title_text="SoH", row=1, col=2)
fig.update_xaxes(title_text="Kilométrage", row=2, col=1)
fig.update_xaxes(title_text="Capacité (kWh)", row=2, col=2)

fig.update_yaxes(title_text="Autonomie réelle (km)", row=1, col=1)
fig.update_yaxes(title_text="Ratio (%)", row=1, col=2)
fig.update_yaxes(title_text="Ratio (%)", row=2, col=1)
fig.update_yaxes(title_text="Ratio (%)", row=2, col=2)

fig.update_layout(
    title_text="Relations clés avec le ratio Autonomie réelle / WLTP",
    height=800,
    showlegend=False
)

fig.show()


## 3. Facteurs d'influence


### 3.1 Influence de la saison


In [ ]:
# Analyse par saison
print("\n=== INFLUENCE DE LA SAISON ===")

stats_season = df_clean.groupby('season').agg({
    'ratio_supposed_wltp': ['mean', 'median', 'std', 'count']
}).round(1)

stats_season = stats_season.reindex(['Hiver', 'Printemps', 'Été', 'Automne'])
print("\nStatistiques par saison :")
print(stats_season)


In [ ]:
# Boxplot par saison
fig = px.box(
    df_clean,
    x='season',
    y='ratio_supposed_wltp',
    title='Impact de la saison sur le ratio Autonomie réelle / WLTP',
    labels={'season': 'Saison', 'ratio_supposed_wltp': 'Ratio (%)'},
    color='season',
    category_orders={'season': ['Hiver', 'Printemps', 'Été', 'Automne']}
)

fig.add_hline(y=100, line_dash="dash", line_color="gray", 
              annotation_text="WLTP (100%)")

fig.update_layout(height=500, showlegend=False)
fig.show()


In [ ]:
# Comparaison par marque et saison
stats_make_season = df_clean.groupby(['make', 'season']).agg({
    'ratio_supposed_wltp': 'mean'
}).reset_index()

fig = px.bar(
    stats_make_season,
    x='make',
    y='ratio_supposed_wltp',
    color='season',
    barmode='group',
    title='Ratio moyen Autonomie réelle / WLTP par marque et saison',
    labels={'make': 'Marque', 'ratio_supposed_wltp': 'Ratio moyen (%)'},
    category_orders={'season': ['Hiver', 'Printemps', 'Été', 'Automne']}
)

fig.add_hline(y=100, line_dash="dash", line_color="gray")
fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

fig.write_html("graph/bar_ratio_wltp_per_make_and_season.html")


### 3.4 Influence du kilométrage (âge)


In [ ]:
# Analyse par kilométrage
print("\n=== INFLUENCE DU KILOMÉTRAGE ===")

# Catégories de kilométrage
df_vin['km_category'] = pd.cut(
    df_vin['ODOMETER_FIRST'],
    bins=[0, 20000, 50000, 100000, float('inf')],
    labels=['0-20k km', '20-50k km', '50-100k km', '>100k km']
)

stats_km = df_vin.groupby('km_category').agg({
    'ratio_supposed_wltp': ['mean', 'median', 'std', 'count'],
    'ODOMETER_FIRST': 'mean'
}).round(1)

print("\nStatistiques par catégorie de kilométrage :")
print(stats_km)


In [ ]:
# Boxplot par kilométrage
fig = px.box(
    df_vin,
    x='km_category',
    y='ratio_supposed_wltp',
    title='Impact du kilométrage sur le ratio Autonomie réelle / WLTP',
    labels={'km_category': 'Kilométrage', 'ratio_supposed_wltp': 'Ratio (%)'},
    color='km_category'
)
fig.add_hline(y=100, line_dash="dash", line_color="gray")
fig.update_layout(height=500, showlegend=False)
fig.show()


## 4. Analyses comparatives


### 4.1 Identification des sur-performants et sous-performants


In [ ]:
# Analyse par modèle avec nombre suffisant d'observations
min_observations = 5

model_stats = df_vin.groupby(['make', 'model_name']).agg({
    'ratio_supposed_wltp': ['mean', 'median', 'std', 'count'],
    'supposed_autonomy': 'mean',
    'wltp_range': 'mean',
    'vin': 'nunique'
}).round(1)

model_stats.columns = ['_'.join(col).strip() for col in model_stats.columns.values]
model_stats = model_stats[model_stats['ratio_supposed_wltp_count'] >= min_observations]

# Tri par ratio moyen
model_stats_sorted = model_stats.sort_values('ratio_supposed_wltp_mean', ascending=False)

print(f"\n=== CLASSEMENT DES MODÈLES (min {min_observations} observations) ===")
print(f"\nNombre de modèles analysés : {len(model_stats_sorted)}")


In [ ]:
df_vin.model_name.value_counts()

In [ ]:
# Top 10 sur-performants
print("\n=== TOP 10 SUR-PERFORMANTS (autonomie réelle proche ou supérieure au WLTP) ===")
top_performers = model_stats_sorted.head(10)
print(top_performers[['ratio_supposed_wltp_mean', 'ratio_supposed_wltp_median', 
                       'supposed_autonomy_mean', 'wltp_range_mean', 'vin_nunique']])


In [ ]:
# Top 10 sous-performants
print("\n=== TOP 10 SOUS-PERFORMANTS (autonomie réelle bien inférieure au WLTP) ===")
bottom_performers = model_stats_sorted.tail(10)
print(bottom_performers[['ratio_supposed_wltp_mean', 'ratio_supposed_wltp_median', 
                         'supposed_autonomy_mean', 'wltp_range_mean', 'vin_nunique']])


In [ ]:
# Visualisation comparative
if len(model_stats_sorted) >= 10:
    top_bottom = pd.concat([
        top_performers.head(5).assign(category='Top 5 sur-performants'),
        bottom_performers.tail(5).assign(category='Top 5 sous-performants')
    ]).reset_index()
    
    top_bottom['model_label'] = top_bottom['make'] + ' - ' + top_bottom['model_name']
    
    fig = px.bar(
        top_bottom,
        y='model_label',
        x='ratio_supposed_wltp_mean',
        color='category',
        title='Sur-performants vs Sous-performants : Ratio Autonomie réelle / WLTP',
        labels={'model_label': 'Modèle', 'ratio_supposed_wltp_mean': 'Ratio moyen (%)'},
        orientation='h',
        color_discrete_map={'Top 5 sur-performants': 'green', 'Top 5 sous-performants': 'red'}
    )
    
    fig.add_vline(x=100, line_dash="dash", line_color="gray", 
                  annotation_text="WLTP (100%)")
    
    fig.update_layout(height=600)
    fig.show()


### 4.2 Comparaison par type de batterie


In [ ]:
# Analyse par type de batterie
if 'battery_chemistry' in df_clean.columns:
    df_battery = df_clean[df_clean['battery_chemistry'].notna()].copy()
    
    if len(df_battery) > 100:
        print("\n=== INFLUENCE DU TYPE DE BATTERIE ===")
        print(f"Données avec type de batterie disponible : {len(df_battery)} phases")
        
        # Types de batteries présents
        print(f"\nTypes de batteries : {df_battery['battery_chemistry'].unique()}")
        
        stats_battery = df_battery.groupby('battery_chemistry').agg({
            'ratio_supposed_wltp': ['mean', 'median', 'std', 'count'],
            'supposed_autonomy': 'mean',
            'wltp_range': 'mean',
            'capacity': 'mean',
            'vin': 'nunique'
        }).round(1)
        
        stats_battery.columns = ['_'.join(col).strip() for col in stats_battery.columns.values]
        stats_battery = stats_battery[stats_battery['ratio_supposed_wltp_count'] >= 30]
        stats_battery = stats_battery.sort_values('ratio_supposed_wltp_mean', ascending=False)
        
        print("\nStatistiques par type de batterie :")
        print(stats_battery)
        
        # Boxplot par type de batterie
        battery_types = stats_battery.index.tolist()
        df_battery_filtered = df_battery[df_battery['battery_chemistry'].isin(battery_types)]
        
        if len(battery_types) > 0:
            fig = px.box(
                df_battery_filtered,
                x='battery_chemistry',
                y='ratio_supposed_wltp',
                title='Impact du type de batterie sur le ratio Autonomie réelle / WLTP',
                labels={'battery_chemistry': 'Type de batterie', 'ratio_supposed_wltp': 'Ratio (%)'},
                color='battery_chemistry'
            )
            fig.add_hline(y=100, line_dash="dash", line_color="gray")
            fig.update_layout(height=500, showlegend=False, xaxis_tickangle=-45)
            fig.show()
        
    else:
        print("\n⚠️ Données de type de batterie insuffisantes")
else:
    print("\n⚠️ Colonne battery_chemistry non disponible")
    
fig.write_html("graph/boxplot_ratio_wltp_per_battery_type.html")


### 4.3 Comparaison par capacité de batterie


In [ ]:
# Analyse par capacité de batterie
if 'capacity' in df_vin.columns:
    df_capacity = df_vin[df_vin['capacity'].notna()].copy()
    
    if len(df_capacity) > 100:
        print("\n=== INFLUENCE DE LA CAPACITÉ DE BATTERIE ===")
        print(f"Données avec capacité disponible : {len(df_capacity)} phases")
        
        # Catégories de capacité
        df_capacity['capacity_category'] = pd.cut(
            df_capacity['capacity'],
            bins=[0, 40, 60, 80, 100, float('inf')],
            labels=['<40 kWh', '40-60 kWh', '60-80 kWh', '80-100 kWh', '>100 kWh']
        )
        
        stats_capacity = df_capacity.groupby('capacity_category').agg({
            'ratio_supposed_wltp': ['mean', 'median', 'std', 'count'],
            'capacity': 'mean',
            'supposed_autonomy': 'mean',
            'wltp_range': 'mean',
            'vin': 'nunique'
        }).round(1)
        
        print("\nStatistiques par catégorie de capacité :")
        print(stats_capacity)
        
        # Boxplot par capacité
        fig = px.box(
            df_capacity,
            x='capacity_category',
            y='ratio_supposed_wltp',
            title='Impact de la capacité batterie sur le ratio Autonomie réelle / WLTP',
            labels={'capacity_category': 'Capacité batterie', 'ratio_supposed_wltp': 'Ratio (%)'},
            color='capacity_category',
            category_orders={'capacity_category': ['<40 kWh', '40-60 kWh', '60-80 kWh', '80-100 kWh', '>100 kWh']}

        )
        fig.add_hline(y=100, line_dash="dash", line_color="gray")
        fig.update_layout(height=500, showlegend=False)
        fig.show()
        fig.write_html("graph/boxplot_ratio_wltp_per_capacity.html")
        
        # Scatter plot capacité vs ratio
        fig = px.scatter(
            df_capacity.sample(n=min(5000, len(df_capacity)), random_state=42),
            x='capacity',
            y='ratio_supposed_wltp',
            title='Relation entre capacité batterie et ratio Autonomie réelle / WLTP',
            labels={'capacity': 'Capacité batterie (kWh)', 'ratio_supposed_wltp': 'Ratio (%)'},
            opacity=0.5,
            color='model_name'
        )
        fig.add_hline(y=100, line_dash="dash", line_color="gray")
        fig.update_layout(height=500)
        fig.show()
        fig.write_html("graph/scatter_ratio_wltp_per_capacity.html")
        
    else:
        print("\n⚠️ Données de capacité insuffisantes")
else:
    print("\n⚠️ Colonne capacity non disponible")
    


## 5. Synthèse et conclusions


In [ ]:
# Synthèse finale
print("\n" + "="*70)
print("SYNTHÈSE DE L'ANALYSE AUTONOMIE RÉELLE VS WLTP")
print("="*70)

print(f"\n1. STATISTIQUES GLOBALES")
print(f"   - Ratio moyen : {df_clean['ratio_supposed_wltp'].mean():.1f}%")
print(f"   - Ratio médian : {df_clean['ratio_supposed_wltp'].median():.1f}%")
print(f"   - Écart au WLTP : {100 - df_clean['ratio_supposed_wltp'].mean():.1f}% en moyenne")
print(f"   - Nombre de phases analysées : {len(df_clean)}")
print(f"   - Nombre de véhicules : {df_clean['vin'].nunique()}")

print(f"\n2. MEILLEURE ET PIRE MARQUE")
best_make = stats_by_make.index[0]
worst_make = stats_by_make.index[-1]
print(f"   - Meilleure : {best_make} ({stats_by_make.loc[best_make, 'Ratio moyen (%)']:.1f}%)")
print(f"   - Pire : {worst_make} ({stats_by_make.loc[worst_make, 'Ratio moyen (%)']:.1f}%)")

if len(model_stats_sorted) > 0:
    print(f"\n3. MODÈLES REMARQUABLES")
    best_model = model_stats_sorted.index[0]
    worst_model = model_stats_sorted.index[-1]
    print(f"   - Sur-performant : {best_model[0]} {best_model[1]} ({model_stats_sorted.iloc[0]['ratio_supposed_wltp_mean']:.1f}%)")
    print(f"   - Sous-performant : {worst_model[0]} {worst_model[1]} ({model_stats_sorted.iloc[-1]['ratio_supposed_wltp_mean']:.1f}%)")

print(f"\n4. FACTEURS D'INFLUENCE IDENTIFIÉS")
print(f"   - Saison : Impact observé (variations saisonnières)")
print(f"   - SoH : Corrélation identifiée avec le ratio")
print(f"   - Kilométrage : Impact sur l'autonomie réelle")

if 'temp_outside_mean' in df_clean.columns and df_clean['temp_outside_mean'].notna().sum() > 100:
    print(f"   - Température : Données disponibles, impact analysé")

if 'trip_type' in df_clean.columns and df_clean['trip_type'].notna().sum() > 100:
    print(f"   - Type de trajet : Données disponibles, impact analysé")

if 'battery_chemistry' in df_clean.columns and df_clean['battery_chemistry'].notna().sum() > 100:
    print(f"   - Type de batterie : Données disponibles, comparaison effectuée")

print("\n" + "="*70)
